In [1]:
%pip install langchain


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
key = os.getenv("GOOGLE_API_KEY")

In [3]:
from langgraph.prebuilt import create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI   
from langchain.tools import tool     


In [4]:
!python3 --version

Python 3.10.11


In [5]:
client = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",google_api_key=key)

In [6]:
from langgraph.prebuilt import create_react_agent
from langgraph_swarm import create_swarm, create_handoff_tool

## Tools for agents


In [7]:
@tool(description="generate_case", return_direct=True)
def generate_case(name: str, date: str) -> str:
    """
    A simple tool that simulates generating an appointment booking case 
    
    Args:
        name (str): The name of the person making the appointment.
        date (str): The date of the appointment.

    Returns:
        str: Confirmation message.
    """
    return "Your case is generated successfully!"

@tool(description="book_appointment", return_direct=True)
def book_appointment(name: str, date: str) -> str:
    """
    A simple tool that simulates booking an appointment.
    
    Args:
        name (str): The name of the person making the appointment.
        date (str): The date of the appointment.
    
    Returns:
        str: Confirmation message.
    """
    return f"Appointment booked for {name} on {date} successfully!"

@tool(description="medicine_info", return_direct=True)
def medicine_info(medicine_name: str) -> str:
    """
    A simple tool that simulates retrieving information about a medicine.
    
    Args:
        medicine_name (str): The name of the medicine.
    
    Returns:
        str: Information about the medicine.
    """
    return f"Information about {medicine_name}: [Details about the medicine]"


## handoff Tools

In [8]:
transfer_to_appointment_scheduler = create_handoff_tool(
    agent_name="appointment_scheduler",
    description="Transfer user to the appointment scheduler.",
)
transfer_to_case_generator = create_handoff_tool(
    agent_name="case_generator",
    description="Transfer user to the case generator.",
)

transfer_to_medicine_inventory = create_handoff_tool(
    agent_name="medicine_inventory",
    description="Transfer user to the medicine inventory assistant.",
)
transfer_to_supervisor = create_handoff_tool(
    agent_name="supervisor",
    description="Transfer user to the supervisor.",
)

## Scheduling Agent

In [9]:
appointment_scheduler = create_react_agent(
    model=client,
    tools=[book_appointment, transfer_to_case_generator, transfer_to_medicine_inventory, transfer_to_supervisor],
    prompt="You are an appointment scheduler agent that can book appointments for users for medical consultations you can also transfer users to the other agents when your part is done.", 
    name="appointment_scheduler"
)

## Case Generator Agent

In [10]:
case_generator = create_react_agent(
    model=client,
    tools=[generate_case, transfer_to_appointment_scheduler, transfer_to_medicine_inventory, transfer_to_supervisor],
    prompt="You are a case generator that can create new cases . You will analyze the user's request and generate a structured case. You can also transfer users to the other agents when your part is done.",
    name="case_generator"
)


## Inventory Management Agent

In [11]:
medicine_inventory = create_react_agent(
    model=client,
    tools=[medicine_info, transfer_to_appointment_scheduler, transfer_to_case_generator, transfer_to_supervisor],
    prompt="You are a medicine inventory assistant that can provide information about medicines. You can also transfer users to the other agents when your part is done.",
    name="medicine_inventory"
)


## Supervisor Agent

In [12]:
supervisor = create_react_agent(
    model=client,
    tools=[transfer_to_appointment_scheduler, transfer_to_case_generator, transfer_to_medicine_inventory],
    prompt="You are a supervisor that can make decisions about which assistant to transfer the user to fullfilling their request. you break the the request down into smaller tasks and decide which assistant is best suited to handle each task." ,
    name="supervisor"
)

## Create Swarm

In [13]:
swarm = create_swarm(
    agents=[appointment_scheduler, case_generator, medicine_inventory, supervisor],
    default_active_agent="supervisor",
    #handoff_tools=[ transfer_to_appointment_scheduler, transfer_to_case_generator, transfer_to_medicine_inventory, transfer_to_supervisor]    
).compile()

In [14]:
from pprint import pprint

In [15]:
for chunk in swarm.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "I need to book an appointment for a patient named John Doe on 2023-10-15. Also, I need to check the availability of Aspirin in the medicine inventory."
            }
        ]
    }
):

    for x in chunk.keys():
     for i in range(len(chunk[x]['messages'])):
        pprint(f"{x} : {chunk[x]['messages'][i]}")
        print('\n')

exception calling callback for <Future at 0x7f10cc577580 state=finished raised ParentCommand>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/concurrent/futures/_base.py", line 342, in _invoke_callbacks
    callback(self)
  File "/home/tejas.raval@simform.dom/Desktop/ViniGraph/pheonix/lib/python3.10/site-packages/langgraph/pregel/runner.py", line 107, in on_done
    self.callback()(task, _exception(fut))  # type: ignore[misc]
  File "/home/tejas.raval@simform.dom/Desktop/ViniGraph/pheonix/lib/python3.10/site-packages/langgraph/pregel/runner.py", line 434, in commit
    raise exception
  File "/home/tejas.raval@simform.dom/Desktop/ViniGraph/pheonix/lib/python3.10/site-packages/langgraph/pregel/executor.py", line 80, in done
    task.result()
  File "/usr/local/lib/python3.10/concurrent/futures/_base.py", line 451, in result
    return self.__get_result()
  File "/usr/local/lib/python3.10/concurrent/futures/_base.py", line 403, in __get_result
    raise self._excepti

("supervisor : content='I need to book an appointment for a patient named John "
 'Doe on 2023-10-15. Also, I need to check the availability of Aspirin in the '
 "medicine inventory.' additional_kwargs={} response_metadata={} "
 "id='cf950e3e-c2be-49ba-8462-b495a15ffb06'")


("supervisor : content='' additional_kwargs={'function_call': {'name': "
 "'transfer_to_medicine_inventory', 'arguments': '{}'}} "
 "response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': "
 "[]}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', "
 "'safety_ratings': []} name='supervisor' "
 "id='run--e6f960e3-4fa6-40fb-a1cd-779213c2ebe9-0' tool_calls=[{'name': "
 "'transfer_to_appointment_scheduler', 'args': {}, 'id': "
 "'550c158d-687b-4ad9-8244-80b074e1397d', 'type': 'tool_call'}, {'name': "
 "'transfer_to_medicine_inventory', 'args': {}, 'id': "
 "'ca6f1552-636c-4063-b624-d63459d723de', 'type': 'tool_call'}] "
 "usage_metadata={'input_tokens': 126, 'output_tokens': 14, 'total_toke

ValueError: Found AIMessages with tool_calls that do not have a corresponding ToolMessage. Here are the first few of those tool calls: [{'name': 'transfer_to_medicine_inventory', 'args': {}, 'id': 'ca6f1552-636c-4063-b624-d63459d723de', 'type': 'tool_call'}].

Every tool call (LLM requesting to call a tool) in the message history MUST have a corresponding ToolMessage (result of a tool invocation to return to the LLM) - this is required by most LLM providers.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_CHAT_HISTORY